In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import struct

# key parameters

x_off = 0
y_off = 4
z_off = 7
time_off = 16
intensity_off = 24
ring_off = 28
returnNum_off = 29
point_step = 32

def extract_point_data(data):
    point_data = {}

    point_data['x'] = struct.unpack('f', data[x_off:x_off + 4])[0]
    point_data['y'] = struct.unpack('f', data[y_off:y_off + 4])[0]
    point_data['z'] = struct.unpack('f', data[z_off:z_off + 4])[0]
    point_data['timestep'] = struct.unpack('d', data[time_off:time_off + 8])[0]
    point_data['intensity'] = struct.unpack('f', data[intensity_off:intensity_off+4])[0]
    point_data['ring'] = struct.unpack('B', data[ring_off:ring_off+1])[0]
    point_data['returnNum'] = struct.unpack('B', data[returnNum_off:returnNum_off+1])[0]

    return point_data

def extract_all_points(data, numPoints):
    all_points = []
    for i in range(numPoints):
        start_index = i * point_step
        point_data = extract_point_data(data[start_index:start_index+point_step])
        all_points.append(point_data)
    return all_points


def extract_velodyne_data_to_csv(velodyne_data):
    df_out = pd.DataFrame(columns=['Time', '1m', '1m_intensity', '1m_return', '10cm', '10cm_intensity', '10cm_return', '1cm', '1cm_intensity', '1cm_return', '1mm', '1mm_intensity', '1mm_return', "100um", '100um_intensity', '100um_return'])
    for i in range(len(velodyne_data)):
        relevant_data = []
        relevant_data.append(datetime.fromtimestamp(velodyne_data.loc[i, 'Time']))
        cloud_data = velodyne_data.loc[i, "data"]
        cloud_bytes = cloud_data.encode('latin1')
        num_points = len(cloud_bytes) // point_step
        all_points = extract_all_points(cloud_bytes, num_points)

        df_all_points = pd.DataFrame(all_points)

        df_all_points['distance'] = df_all_points.apply(lambda row: np.sqrt((row.x ** 2) + (row.y ** 2) + (row.z ** 2)),
                                                        axis=1)

        for j in range(5):
            subset = df_all_points.loc[df_all_points['distance'] <= (10 ** (-1 * j))]
            loading = len(subset)
            relevant_data.append(loading)
            intensity = subset['intensity'].mean()
            relevant_data.append(intensity)
            returnNum = subset['returnNum'].mean()
            relevant_data.append(returnNum)
            #relevant_data.append([loading, intensity, returnNum])
        df_out.loc[len(df_out)] = relevant_data
    return df_out


In [2]:
import glob
from bagpy import bagreader

hovermap_files = glob.glob('DeclineFeb2025_01/*.bag')
count = 1
for filename in hovermap_files:
        b = bagreader(filename)
        data = b.message_by_topic('/velodyne_points')
        df_data = pd.read_csv(data)
        df_out = extract_velodyne_data_to_csv(df_data)
        df_out.to_csv("DeclineFeb2025_01/hovermap{}.csv".format(count))
        count += 1
if not hovermap_files:
    print('empty')

[INFO]  Successfully created the data folder DeclineFeb2025_01\DeclineFeb2025_2025-02-05-15-28-56_0.
[INFO]  Successfully created the data folder DeclineFeb2025_01\DeclineFeb2025_2025-02-05-15-33-40_1.
[INFO]  Successfully created the data folder DeclineFeb2025_01\DeclineFeb2025_2025-02-05-15-37-45_2.
[INFO]  Successfully created the data folder DeclineFeb2025_01\DeclineFeb2025_2025-02-05-15-41-49_3.


In [3]:
import glob
hovermap_csvs = glob.glob('DeclineFeb2025_01/hovermap*.csv')

dfs = []
for filename in hovermap_csvs:
    data = pd.read_csv(filename)
    data = data.drop(data.columns[0], axis=1)
    dfs.append(data)
hovermap_data = pd.concat(dfs)

In [4]:
hovermap_data.sort_values(by='Time', inplace=True)

hovermap_data['1m_diff'] = hovermap_data['1m'].diff()
hovermap_data['10cm_diff'] = hovermap_data['10cm'].diff()
hovermap_data['1cm_diff'] = hovermap_data['1cm'].diff()
hovermap_data['1mm_diff'] = hovermap_data['1mm'].diff()
hovermap_data['100um_diff'] = hovermap_data['100um'].diff()
hovermap_data['Timestamp'] = pd.to_datetime(hovermap_data['Time'])


In [5]:
hovermap_data.to_csv('DeclineFeb2025_01/combindata.csv')